In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import os, sys
import random, math, time
import shutil
import numpy as np
# import matplotlib.pyplot as plt

In [2]:
LEARNING_RATE = 0.001
MOMENTUM = 0.9
EPOCHS = 2 
BATCH_SIZE = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.ImageFolder('/data/Imagenet/train',transform=transform)
testset = torchvision.datasets.ImageFolder('/data/Imagenet/val',transform=transform)

loaders = {
    'train' : torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True),
    'test'  : torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False),
}

total_train_batch = len(loaders['train'])
total_test_batch = len(loaders['test'])
print('train 총 배치의 수 : {}'.format(total_train_batch))
print('test 총 배치의 수 : {}'.format(total_test_batch))

train 총 배치의 수 : 160146
test 총 배치의 수 : 6250


In [4]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        print("device:", torch.cuda.get_device_name(torch.cuda.current_device()))
    else:
        device = torch.device("cpu")
        print("device: cpu")
    return device
device = get_device()

device: GeForce RTX 2060 SUPER


In [5]:
def convert_to_imshow_format(image):
    # first convert back to [0,1] range from [-1,1] range
    image = image / 2 + 0.5
    image = image.numpy()
    # convert from CHW to HWC
    # from 3x32x32 to 32x32x3
    return image.transpose(1,2,0)

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'], valid_loss_min

def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0
    weights = torch.zeros(4, 10, 64)  # [b, 10, hid_dim]
    for idx, (inputs, targets) in enumerate(iterator):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
#         weights = model.state_dict()['attention.fc_k.weight']

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = outputs.max(1)                  # probs, index = torch.max(outputs.data, dim=1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()  # correct += (predicted == labels).sum().item()

    return epoch_loss/len(iterator), 100.*correct/total

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    correct = 0
    total = 0
    weights = torch.zeros(4, 10, 64) # [b, 10, hid_dim]
    with torch.no_grad():
        for idx, (inputs, targets) in enumerate(iterator):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
#             weights = model.state_dict()['attention.fc_k.weight']

            loss = criterion(outputs, targets)

            epoch_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return epoch_loss/len(iterator), 100.*correct/total

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [6]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            #3 224 224
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            #64 112 112
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            #128 56 56
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            #256 28 28
            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            #512 14 14
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
            #512 7 7
        )
        

#         self.classifier = nn.Sequential(
#             nn.Linear(512, 4096),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(4096, 1000),
#         )
        
    
        self.avg_pool = nn.AvgPool2d(7)
        self.classifier = nn.Linear(512, 4096)
    
    
    def forward(self, x):
        x = self.features(x)
        
        x = self.avg_pool(x)
        
        x = x.view(x.size(0),-1)

        x = self.classifier(x)
        
        return x
    

In [7]:
# 모델 정의하기
model = VGG().to(device)
print(model)

# 손실 함수와 Optimizer 정의하기
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# In[ ]: train
checkpoint_path = './checkpoint/{}_model_{}_epoch_{}.pth'.format('ImageNet', 'VGG', EPOCHS)
best_model_path = './checkpoint/{}_best_model_{}_epoch_{}.pth'.format('ImageNet', 'VGG', EPOCHS)
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []
best_valid_loss = float('inf')

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [8]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [9]:
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    epoch_loss = AverageMeter()

    if(epoch>0):
        net = Net()
        net.load_state_dict(torch.load(save_path))
        net.to(device)
    start_time = time.time()
    for i, data in enumerate(loaders['train'], 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss.update(loss.item())
        
        if i % 1000 == 0:
            end_time=time.time()
            Min,Sec=epoch_time(start_time,end_time)
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                      epoch, i, len(loaders['train']), loss=epoch_loss))
            print("Time : {}분 {}초".format(Min,Sec))
            start_time = time.time()
            
    print('Finished Training')

    model.eval()
    epoch_loss = AverageMeter()
    correct = 0
    total = 0

    with torch.no_grad():
         for i, data in enumerate(loaders['train'], 0):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = model(images)
            
            epoch_loss.update(loss.item())
            
            if i % 1000 == 0:
                print('Tarest: [{0}/{1}]\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                          i, len(testloader), loss=epoch_loss))

Epoch: [0][0/160146]	Loss 8.5142 (8.5142)	
Time : 0분 1초
Epoch: [0][1000/160146]	Loss 7.2129 (7.4810)	
Time : 5분 10초
Epoch: [0][2000/160146]	Loss 6.9573 (7.2425)	
Time : 5분 38초
Epoch: [0][3000/160146]	Loss 6.8727 (7.1422)	
Time : 5분 32초
Epoch: [0][4000/160146]	Loss 6.8958 (7.0816)	
Time : 5분 18초
Epoch: [0][5000/160146]	Loss 7.2691 (7.0362)	
Time : 5분 47초
Epoch: [0][6000/160146]	Loss 7.0663 (7.0016)	
Time : 5분 22초
Epoch: [0][7000/160146]	Loss 6.8435 (6.9753)	
Time : 5분 31초
Epoch: [0][8000/160146]	Loss 6.6790 (6.9512)	
Time : 5분 33초
Epoch: [0][9000/160146]	Loss 6.7137 (6.9320)	
Time : 5분 35초
Epoch: [0][10000/160146]	Loss 6.8792 (6.9141)	
Time : 5분 31초
Epoch: [0][11000/160146]	Loss 6.6282 (6.8972)	
Time : 5분 33초
Epoch: [0][12000/160146]	Loss 6.7025 (6.8822)	
Time : 5분 58초
Epoch: [0][13000/160146]	Loss 6.7529 (6.8655)	
Time : 5분 16초
Epoch: [0][14000/160146]	Loss 6.8662 (6.8491)	
Time : 5분 34초
Epoch: [0][15000/160146]	Loss 6.6120 (6.8308)	
Time : 5분 42초
Epoch: [0][16000/160146]	Loss 6.6468 (

/home/linux/anaconda3/envs/Pycharm/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [0][27000/160146]	Loss 5.9878 (6.6017)	
Time : 5분 50초
Epoch: [0][28000/160146]	Loss 6.4940 (6.5843)	
Time : 5분 34초
Epoch: [0][29000/160146]	Loss 5.4503 (6.5675)	
Time : 5분 35초
Epoch: [0][30000/160146]	Loss 6.9102 (6.5515)	
Time : 5분 48초
Epoch: [0][31000/160146]	Loss 5.9455 (6.5356)	
Time : 5분 29초
Epoch: [0][32000/160146]	Loss 5.0474 (6.5197)	
Time : 5분 43초
Epoch: [0][33000/160146]	Loss 6.2061 (6.5043)	
Time : 5분 32초
Epoch: [0][34000/160146]	Loss 6.4356 (6.4887)	
Time : 5분 30초
Epoch: [0][35000/160146]	Loss 6.2197 (6.4726)	
Time : 5분 42초
Epoch: [0][36000/160146]	Loss 5.5562 (6.4574)	
Time : 5분 37초
Epoch: [0][37000/160146]	Loss 5.6271 (6.4426)	
Time : 5분 36초
Epoch: [0][38000/160146]	Loss 5.6325 (6.4266)	
Time : 5분 18초
Epoch: [0][39000/160146]	Loss 6.2774 (6.4107)	
Time : 5분 38초
Epoch: [0][40000/160146]	Loss 5.9679 (6.3953)	
Time : 5분 31초
Epoch: [0][41000/160146]	Loss 5.2668 (6.3793)	
Time : 5분 41초
Epoch: [0][42000/160146]	Loss 5.3896 (6.3635)	
Time : 5분 34초
Epoch: [0][43000/160146]

Finished Training


NameError: name 'testloader' is not defined

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(),lr=0.001)

In [ ]:
def epoch_time(start_time, end_time):
    time_gap = end_time - start_time
    time_min = int(time_gap/60)
    time_sec = int(time_gap-(time_min*60))
    return time_min,time_sec

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times
    epoch_loss = AverageMeter()

    if(epoch>0):
        net = Net()
        net.load_state_dict(torch.load(save_path))
        net.to(device)
    start_time = time.time()
    for i, data in enumerate(loaders['train'], 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs,f = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss.update(loss.item())
        
        if i % 1000 == 0:
            end_time=time.time()
            Min,Sec=epoch_time(start_time,end_time)
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                      epoch, i, len(trainloader), loss=epoch_loss))
            print("Time : {}분 {}초".format(Min,Sec))
            start_time = time.time()
            
    print('Finished Training')

    model.eval()
    epoch_loss = AverageMeter()
    correct = 0
    total = 0

    with torch.no_grad():
         for i, data in enumerate(trainloader, 0):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)
            
            epoch_loss.update(loss.item())
            
            if i % 1000 == 0:
                print('Tarest: [{0}/{1}]\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                          i, len(testloader), loss=epoch_loss))

In [ ]:
Epoch: [0][0/160146]	Loss 6.0523 (6.0523)	
Time : 0분 0초
Epoch: [0][1000/160146]	Loss 6.2118 (6.1990)	
Time : 4분 49초
Epoch: [0][2000/160146]	Loss 5.6622 (6.1689)	
Time : 5분 33초
Epoch: [0][3000/160146]	Loss 6.0832 (6.1522)	
Time : 5분 14초
Epoch: [0][4000/160146]	Loss 6.0557 (6.1371)	
Time : 5분 41초
Epoch: [0][5000/160146]	Loss 6.1596 (6.1240)	
Time : 5분 8초
Epoch: [0][6000/160146]	Loss 6.4108 (6.1180)	
Time : 5분 42초
Epoch: [0][7000/160146]	Loss 5.8664 (6.1117)	
Time : 5분 25초
Epoch: [0][8000/160146]	Loss 5.1508 (6.1037)	
Time : 5분 35초
Epoch: [0][9000/160146]	Loss 5.1602 (6.0949)	
Time : 5분 27초
Epoch: [0][10000/160146]	Loss 5.7366 (6.0865)	
Time : 5분 25초
Epoch: [0][11000/160146]	Loss 5.9805 (6.0790)	
Time : 5분 39초
Epoch: [0][12000/160146]	Loss 5.6696 (6.0717)	
Time : 5분 26초
Epoch: [0][13000/160146]	Loss 5.9098 (6.0650)	
Time : 5분 41초
Epoch: [0][14000/160146]	Loss 5.6462 (6.0581)	
Time : 5분 25초
Epoch: [0][15000/160146]	Loss 5.8114 (6.0507)	
Time : 5분 46초
Epoch: [0][16000/160146]	Loss 6.0732 (6.0437)	
Time : 5분 27초
Epoch: [0][17000/160146]	Loss 6.0056 (6.0345)	
Time : 5분 44초
Epoch: [0][18000/160146]	Loss 5.1952 (6.0280)	
Time : 5분 31초
Epoch: [0][19000/160146]	Loss 5.9679 (6.0205)	
Time : 5분 41초
Epoch: [0][20000/160146]	Loss 6.0163 (6.0139)	
Time : 5분 33초
Epoch: [0][21000/160146]	Loss 5.7392 (6.0077)	
Time : 5분 35초
Epoch: [0][22000/160146]	Loss 5.3455 (5.9999)	
Time : 5분 40초
Epoch: [0][23000/160146]	Loss 5.9221 (5.9935)	
Time : 5분 26초
Epoch: [0][24000/160146]	Loss 6.2538 (5.9871)	
Time : 5분 40초
Epoch: [0][25000/160146]	Loss 5.4785 (5.9812)	
Time : 5분 33초
Epoch: [0][26000/160146]	Loss 5.2333 (5.9756)	
Time : 5분 35초
Epoch: [0][27000/160146]	Loss 6.1026 (5.9701)	
Time : 5분 34초
Epoch: [0][28000/160146]	Loss 5.7497 (5.9647)	
Time : 5분 39초
Epoch: [0][29000/160146]	Loss 5.3987 (5.9588)	
Time : 5분 39초
Epoch: [0][30000/160146]	Loss 5.3904 (5.9532)	
Time : 5분 31초
Epoch: [0][31000/160146]	Loss 6.2879 (5.9470)	
Time : 5분 43초
Epoch: [0][32000/160146]	Loss 5.2256 (5.9415)	
Time : 5분 33초
Epoch: [0][33000/160146]	Loss 5.3170 (5.9355)	
Time : 5분 39초
Epoch: [0][34000/160146]	Loss 4.6125 (5.9302)	
Time : 5분 27초
Epoch: [0][35000/160146]	Loss 6.0057 (5.9247)	
Time : 5분 41초
Epoch: [0][36000/160146]	Loss 5.7322 (5.9191)	
Time : 5분 48초
Epoch: [0][37000/160146]	Loss 5.1329 (5.9138)	
Time : 5분 41초
Epoch: [0][38000/160146]	Loss 5.5605 (5.9087)	
Time : 5분 59초
Epoch: [0][39000/160146]	Loss 5.7525 (5.9040)	
Time : 5분 42초
Epoch: [0][40000/160146]	Loss 5.5179 (5.8978)	
Time : 5분 51초
Epoch: [0][41000/160146]	Loss 5.6194 (5.8923)	
Time : 5분 38초
Epoch: [0][42000/160146]	Loss 5.6479 (5.8873)	
Time : 5분 28초
Epoch: [0][43000/160146]	Loss 5.7095 (5.8829)	
Time : 5분 37초
Epoch: [0][44000/160146]	Loss 5.5837 (5.8779)	
Time : 5분 33초
Epoch: [0][45000/160146]	Loss 5.5681 (5.8737)	
Time : 5분 39초
Epoch: [0][46000/160146]	Loss 6.1438 (5.8690)	
Time : 5분 29초
Epoch: [0][47000/160146]	Loss 5.3175 (5.8632)	
Time : 5분 40초
Epoch: [0][48000/160146]	Loss 5.5360 (5.8575)	
Time : 5분 28초
Epoch: [0][49000/160146]	Loss 5.3911 (5.8517)	
Time : 5분 46초
Epoch: [0][50000/160146]	Loss 5.0863 (5.8469)	
Time : 5분 24초
Epoch: [0][51000/160146]	Loss 5.7879 (5.8420)	
Time : 5분 40초
Epoch: [0][52000/160146]	Loss 5.7161 (5.8376)	
Time : 5분 28초
Epoch: [0][53000/160146]	Loss 5.4223 (5.8332)	
Time : 5분 42초
Epoch: [0][54000/160146]	Loss 5.1143 (5.8285)	
Time : 5분 36초
Epoch: [0][55000/160146]	Loss 5.5925 (5.8234)	
Time : 5분 37초
Epoch: [0][56000/160146]	Loss 5.3371 (5.8183)	
Time : 5분 37초
Epoch: [0][57000/160146]	Loss 5.0578 (5.8133)	
Time : 5분 38초
Epoch: [0][58000/160146]	Loss 5.9369 (5.8088)	
Time : 5분 36초
Epoch: [0][59000/160146]	Loss 5.6628 (5.8045)	
Time : 5분 33초
Epoch: [0][60000/160146]	Loss 5.9631 (5.7997)	
Time : 5분 38초
Epoch: [0][61000/160146]	Loss 5.0647 (5.7946)	
Time : 5분 29초
Epoch: [0][62000/160146]	Loss 5.6345 (5.7899)	
Time : 5분 43초
Epoch: [0][63000/160146]	Loss 5.1194 (5.7850)	
Time : 5분 37초
Epoch: [0][64000/160146]	Loss 6.1338 (5.7806)	
Time : 5분 35초
Epoch: [0][65000/160146]	Loss 6.3280 (5.7767)	
Time : 5분 46초
Epoch: [0][66000/160146]	Loss 4.6789 (5.7722)	
Time : 5분 37초
Epoch: [0][67000/160146]	Loss 5.9478 (5.7678)	
Time : 5분 39초
Epoch: [0][68000/160146]	Loss 5.7040 (5.7636)	
Time : 5분 28초
Epoch: [0][69000/160146]	Loss 6.1802 (5.7591)	
Time : 5분 33초
Epoch: [0][70000/160146]	Loss 5.2994 (5.7547)	
Time : 5분 41초
Epoch: [0][71000/160146]	Loss 5.1728 (5.7502)	
Time : 5분 41초
Epoch: [0][72000/160146]	Loss 5.3269 (5.7459)	
Time : 5분 30초
Epoch: [0][73000/160146]	Loss 5.3739 (5.7418)	
Time : 5분 49초
Epoch: [0][74000/160146]	Loss 4.9992 (5.7377)	
Time : 5분 32초
Epoch: [0][75000/160146]	Loss 4.7486 (5.7337)	
Time : 5분 42초
Epoch: [0][76000/160146]	Loss 5.2529 (5.7289)	
Time : 5분 31초
Epoch: [0][77000/160146]	Loss 5.6053 (5.7248)	
Time : 5분 42초
Epoch: [0][78000/160146]	Loss 4.4105 (5.7205)	
Time : 5분 41초
Epoch: [0][79000/160146]	Loss 5.1643 (5.7163)	
Time : 5분 33초
Epoch: [0][80000/160146]	Loss 5.6621 (5.7126)	
Time : 5분 40초
Epoch: [0][81000/160146]	Loss 5.4557 (5.7083)	
Time : 5분 34초
Epoch: [0][82000/160146]	Loss 5.5852 (5.7041)	
Time : 5분 36초
Epoch: [0][83000/160146]	Loss 3.9882 (5.7001)	
Time : 5분 27초
Epoch: [0][84000/160146]	Loss 5.0201 (5.6958)	
Time : 5분 37초
Epoch: [0][85000/160146]	Loss 5.5833 (5.6913)	
Time : 5분 39초
Epoch: [0][86000/160146]	Loss 4.3062 (5.6870)	
Time : 5분 32초
Epoch: [0][87000/160146]	Loss 4.4862 (5.6828)	
Time : 5분 38초
Epoch: [0][88000/160146]	Loss 4.1190 (5.6783)	
Time : 5분 34초
Epoch: [0][89000/160146]	Loss 5.1969 (5.6742)	
Time : 5분 45초
Epoch: [0][90000/160146]	Loss 5.6429 (5.6700)	
Time : 5분 43초
Epoch: [0][91000/160146]	Loss 5.3844 (5.6658)	
Time : 5분 31초
Epoch: [0][92000/160146]	Loss 5.2150 (5.6616)	
Time : 5분 32초
Epoch: [0][93000/160146]	Loss 4.3356 (5.6573)	
Time : 5분 40초
Epoch: [0][94000/160146]	Loss 4.9228 (5.6530)	
Time : 5분 44초
Epoch: [0][95000/160146]	Loss 5.5327 (5.6492)	
Time : 5분 35초
Epoch: [0][96000/160146]	Loss 4.5716 (5.6454)	
Time : 5분 46초
Epoch: [0][97000/160146]	Loss 5.6319 (5.6414)	
Time : 5분 28초
Epoch: [0][98000/160146]	Loss 5.0139 (5.6374)	
Time : 5분 42초
Epoch: [0][99000/160146]	Loss 5.0655 (5.6332)	
Time : 5분 40초
Epoch: [0][100000/160146]	Loss 4.7182 (5.6296)	
Time : 5분 37초
Epoch: [0][101000/160146]	Loss 3.5861 (5.6256)	
Time : 5분 34초
Epoch: [0][102000/160146]	Loss 5.6979 (5.6219)	
Time : 5분 42초
Epoch: [0][103000/160146]	Loss 5.1209 (5.6178)	
Time : 5분 45초
Epoch: [0][104000/160146]	Loss 5.9400 (5.6138)	
Time : 5분 45초
Epoch: [0][105000/160146]	Loss 5.5711 (5.6099)	
Time : 5분 35초
Epoch: [0][106000/160146]	Loss 4.8142 (5.6058)	
Time : 5분 41초
Epoch: [0][107000/160146]	Loss 4.8170 (5.6021)	
Time : 5분 33초
Epoch: [0][108000/160146]	Loss 5.6805 (5.5982)	
Time : 5분 51초
Epoch: [0][109000/160146]	Loss 4.8792 (5.5945)	
Time : 5분 39초
Epoch: [0][110000/160146]	Loss 4.2426 (5.5905)	
Time : 5분 33초
Epoch: [0][111000/160146]	Loss 5.1861 (5.5865)	
Time : 5분 58초
Epoch: [0][112000/160146]	Loss 5.3117 (5.5827)	
Time : 5분 34초
Epoch: [0][113000/160146]	Loss 6.3168 (5.5790)	
Time : 5분 51초
Epoch: [0][114000/160146]	Loss 5.1036 (5.5752)	
Time : 5분 44초
Epoch: [0][115000/160146]	Loss 5.6217 (5.5715)	
Time : 5분 34초
Epoch: [0][116000/160146]	Loss 5.7410 (5.5676)	
Time : 5분 46초
Epoch: [0][117000/160146]	Loss 4.5554 (5.5640)	
Time : 5분 38초
Epoch: [0][118000/160146]	Loss 4.7372 (5.5599)	
Time : 5분 43초
Epoch: [0][119000/160146]	Loss 6.4589 (5.5564)	
Time : 5분 48초
Epoch: [0][120000/160146]	Loss 5.0898 (5.5524)	
Time : 5분 36초
Epoch: [0][121000/160146]	Loss 5.2419 (5.5485)	
Time : 5분 50초
Epoch: [0][122000/160146]	Loss 4.6553 (5.5444)	
Time : 5분 53초
Epoch: [0][123000/160146]	Loss 4.8555 (5.5407)	
Time : 5분 25초
Epoch: [0][124000/160146]	Loss 4.1028 (5.5372)	
Time : 5분 57초
Epoch: [0][125000/160146]	Loss 5.6457 (5.5330)	
Time : 5분 40초
Epoch: [0][126000/160146]	Loss 5.5441 (5.5294)	
Time : 5분 44초
Epoch: [0][127000/160146]	Loss 5.2375 (5.5258)	
Time : 5분 53초
Epoch: [0][128000/160146]	Loss 4.6963 (5.5219)	
Time : 5분 41초
Epoch: [0][129000/160146]	Loss 5.4414 (5.5183)	
Time : 5분 43초
Epoch: [0][130000/160146]	Loss 5.5871 (5.5148)	
Time : 5분 52초
Epoch: [0][131000/160146]	Loss 5.2440 (5.5109)	
Time : 5분 46초
Epoch: [0][132000/160146]	Loss 4.8877 (5.5071)	
Time : 5분 42초
Epoch: [0][133000/160146]	Loss 5.7910 (5.5031)	
Time : 5분 50초

Epoch: [0][134000/160146]	Loss 4.5534 (5.4993)	
Time : 5분 46초
Epoch: [0][135000/160146]	Loss 6.0764 (5.4955)	
Time : 5분 47초
Epoch: [0][136000/160146]	Loss 4.2074 (5.4917)	
Time : 5분 37초
Epoch: [0][137000/160146]	Loss 5.7228 (5.4878)	
Time : 5분 52초
Epoch: [0][138000/160146]	Loss 4.9297 (5.4842)	
Time : 5분 35초
Epoch: [0][139000/160146]	Loss 4.3944 (5.4805)	
Time : 5분 58초
Epoch: [0][140000/160146]	Loss 4.7563 (5.4766)	
Time : 5분 34초
Epoch: [0][141000/160146]	Loss 4.5714 (5.4729)	
Time : 5분 47초
Epoch: [0][142000/160146]	Loss 5.6726 (5.4695)	
Time : 5분 54초
Epoch: [0][143000/160146]	Loss 5.3255 (5.4658)	
Time : 5분 47초
Epoch: [0][144000/160146]	Loss 5.2520 (5.4621)	
Time : 5분 45초
Epoch: [0][145000/160146]	Loss 4.6506 (5.4586)	
Time : 5분 46초
Epoch: [0][146000/160146]	Loss 5.5129 (5.4551)	
Time : 6분 0초
Epoch: [0][147000/160146]	Loss 4.5918 (5.4515)	
Time : 5분 44초
Epoch: [0][148000/160146]	Loss 4.1942 (5.4479)	
Time : 5분 41초
Epoch: [0][149000/160146]	Loss 5.2236 (5.4444)	
Time : 5분 44초
Epoch: [0][150000/160146]	Loss 5.1904 (5.4409)	
Time : 5분 47초
Epoch: [0][151000/160146]	Loss 4.8111 (5.4374)	
Time : 5분 48초
Epoch: [0][152000/160146]	Loss 5.4540 (5.4339)	
Time : 5분 42초
Epoch: [0][153000/160146]	Loss 6.3098 (5.4304)	
Time : 5분 49초
Epoch: [0][154000/160146]	Loss 5.3532 (5.4269)	
Time : 5분 49초
Epoch: [0][155000/160146]	Loss 5.1984 (5.4235)	
Time : 5분 48초
Epoch: [0][156000/160146]	Loss 4.8036 (5.4198)	
Time : 5분 48초
Epoch: [0][157000/160146]	Loss 4.2983 (5.4166)	
Time : 5분 46초
Epoch: [0][158000/160146]	Loss 5.3792 (5.4133)	
Time : 5분 53초
Epoch: [0][159000/160146]	Loss 6.1380 (5.4098)	
Time : 5분 52초
Epoch: [0][160000/160146]	Loss 4.7154 (5.4063)	
Time : 5분 51초
Finished Training
